In [3]:
!pip install weaviate-client
!pip install datasets

/Users/jb/miniconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=5875) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [16]:
# declare name of the collection
COLLECTION_NAME = "QuestionAnswering_No_Korean_Tokenizer"
QUERY_SENTENCE = "사이보그가 뭐야?"
NUM_OF_DATA = 200
WEIGHT_OF_VECTOR_IN_HYBRID = 0.5

In [26]:
import pandas as pd
from datasets import load_dataset

DATA_SET    = "beomi/KoAlpaca-v1.1a"
DATA_FILE   = "./data/KoAlpaca-train.csv"

load_data = load_dataset(DATA_SET, split="train")
load_data.to_csv(DATA_FILE)
csv_data = pd.read_csv(DATA_FILE)
data_to_insert = csv_data.head(min(NUM_OF_DATA, len(load_data)))
data_to_insert.sample(5)

Creating CSV from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

,instruction,output,url
70,모기는 이산화탄소를 어떻게 감지하고 어떤 고성능 센서 기능을 갖추고 있나요?,"암컷 모기는 냄새, 이산화탄소, 열, 시각정보 등을 통해 목표를 추적하고 흡혈을 합...",https://kin.naver.com/qna/detail.naver?d1id=11...
14,조선시대에 성인 나이는 몇 살이었나요? 성인식도 있었나요?,조선시대에 성인 나이는 15세였습니다. 관례라는 성인식이 있었습니다. 관례는 남성의...,https://kin.naver.com/qna/detail.naver?d1id=11...
84,중국 화폐에는 소수민족이 그려져 있는 화폐도 있는데 그 소수민족이 누구인가요? 그리...,중국의 지폐와 동전에는 한국의 원화와 마찬가지로 다양한 인물이 그려져 있습니다. 중...,https://kin.naver.com/qna/detail.naver?d1id=6&...
108,지렁이는 알을 낳을까요? 그냥 지렁이 새끼를 그냥 낳아버릴까요? 지렁이가 왜 흙을 ...,체절(몸의 마디)이 있는 원통형의 길쭉한 몸을 가진 지렁이는 생식을 할 때는 다른 ...,https://kin.naver.com/qna/detail.naver?d1id=13...
105,왜 로켓을 발사할 때 바다에서 쏘는 것일까요?,"바다에서 로켓을 쏘는 이유는 여러 가지가 있습니다. 첫째로, 경제적인 이유가 있습니...",https://kin.naver.com/qna/detail.naver?d1id=11...


In [27]:
import weaviate

client = weaviate.connect_to_local(host='localhost', port=8080)

/Users/jb/miniconda3/lib/python3.12/site-packages/weaviate/warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


In [28]:
import weaviate.classes.config as wc

# Drop the collection
client.collections.delete(name=COLLECTION_NAME)

client.collections.create(
    name=COLLECTION_NAME,
    description=COLLECTION_NAME,
    properties=[
        wc.Property(
            name="instruction",
            data_type=wc.DataType.TEXT,
            # tokenization=Tokenization.KAGOME_KR
        ),
        wc.Property(
            name="output",
            data_type=wc.DataType.TEXT,
            # tokenization=Tokenization.KAGOME_KR
        ),
        wc.Property(
            name="url",
            data_type=wc.DataType.TEXT,
            skip_vectorization=True),
    ],
    vectorizer_config=wc.Configure.Vectorizer.text2vec_transformers(),
)

In [29]:
from weaviate.collections.classes.filters import Filter

target_collection = client.collections.get(COLLECTION_NAME)

# Fetch the first 200 rows from csv_data
data_to_insert = csv_data.head(min(NUM_OF_DATA, len(csv_data)))

with target_collection.batch.dynamic() as batch:
    for _, row in data_to_insert.iterrows():
        # 중복 확인
        response = target_collection.query.fetch_objects(
            filters=Filter.by_property("instruction").equal(row["instruction"]),
            limit=1
        )
        # object가 있으면 건너뛰기
        if response.objects:
            continue

        batch.add_object(
            properties={
                "instruction": row["instruction"],
                "output": row["output"],
                "url": row["url"],
            },
        )


In [30]:
from weaviate.classes.query import MetadataQuery

target_collection = client.collections.get(COLLECTION_NAME)

response = target_collection.query.near_text(
    query=QUERY_SENTENCE,
    limit=3,
    return_metadata=MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.distance)

{'instruction': "'천추의 한'이 무슨 뜻인가요? 궁금합니다.", 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=11080102&docId=46686890', 'output': "'천추의 한'은 '오랜 시간, 먼 미래'를 뜻하는 '천추'와 '한' 즉, '한(한강)을 품고 살아가다 말하지 못한 아쉬움'을 뜻하는 '한'이 결합된 단어입니다. 한(한강)을 풀지 못하고 죽는 사람이 주로 내뱉는 말 중에 “그 놈을 죽이지 못한 것이 천추의 한이다.” 이런 대사가 많은데, 여기서 천추는 죽을 때까지, 평생 동안 자신을 괴롭혀 왔던 한(한강을 풀지 못한 한)이기에 뼈에 사무친다는 의미로 사용되는 것입니다. '천추의 한'은 오랜 시간 동안 쌓이고, 해결하지 못한 아쉬움이라는 뜻으로 사용됩니다."}
0.3401041030883789
{'instruction': '"데요"와 "대요"의 차이점은 무엇인가요? "이상한데?"나 "아닌데?" 같은 경우도 포함되나요? 뉴스를 보고 나서 "내일 비 온다"라면 "내일 비 온대"와 "내일 비 온데"라는 표현의 차이점은 무엇인가요? "어째 이상하데?" 같은 경우에는 어떤 상황에서 사용되나요?', 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=11080103&docId=275232662', 'output': '"-대"를 "-다고 해"를 줄인 것이라는 것을 이해하는 것이 중요합니다. "-다고 해"와 "-대"는 어떤 것을 표현할 때 사용하는 것이기 때문에, "-대"는 자신이 본 것을 말할 때, "-다"는 타인의 언급을 전달할 때 주로 사용됩니다. 따라서 "어째 이상하데?"와 같이 직접적인 언급을 할 때도 "-데"를 사용합니다. \n\n그리고 "이상한데?"나 "아닌데?"와 같은 경우, "이상하다"나 "아니다"와 같은 형용사가 바로 앞에 나와 있기 때문에 "-ㄴ데"라는 어미를 사용합니다. 또한 뉴스를 보고 "내일 비 

In [31]:
target_collection = client.collections.get(COLLECTION_NAME)

response = target_collection.query.bm25(
    query=QUERY_SENTENCE,
    return_metadata=MetadataQuery(score=True),
    limit=3
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.score)

In [32]:
target_collection = client.collections.get(COLLECTION_NAME)

response = target_collection.query.hybrid(
    query=QUERY_SENTENCE,
    alpha=WEIGHT_OF_VECTOR_IN_HYBRID,
    return_metadata=MetadataQuery(score=True, explain_score=True),
    limit=3,
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.score, o.metadata.explain_score)

{'instruction': "'천추의 한'이 무슨 뜻인가요? 궁금합니다.", 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=11080102&docId=46686890', 'output': "'천추의 한'은 '오랜 시간, 먼 미래'를 뜻하는 '천추'와 '한' 즉, '한(한강)을 품고 살아가다 말하지 못한 아쉬움'을 뜻하는 '한'이 결합된 단어입니다. 한(한강)을 풀지 못하고 죽는 사람이 주로 내뱉는 말 중에 “그 놈을 죽이지 못한 것이 천추의 한이다.” 이런 대사가 많은데, 여기서 천추는 죽을 때까지, 평생 동안 자신을 괴롭혀 왔던 한(한강을 풀지 못한 한)이기에 뼈에 사무친다는 의미로 사용되는 것입니다. '천추의 한'은 오랜 시간 동안 쌓이고, 해결하지 못한 아쉬움이라는 뜻으로 사용됩니다."}
0.5 
Hybrid (Result Set vector,hybridVector) Document b67defeb-688b-4523-b41a-234c30797ded: original score 0.6598959, normalized score: 0.5
{'instruction': '"데요"와 "대요"의 차이점은 무엇인가요? "이상한데?"나 "아닌데?" 같은 경우도 포함되나요? 뉴스를 보고 나서 "내일 비 온다"라면 "내일 비 온대"와 "내일 비 온데"라는 표현의 차이점은 무엇인가요? "어째 이상하데?" 같은 경우에는 어떤 상황에서 사용되나요?', 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=11080103&docId=275232662', 'output': '"-대"를 "-다고 해"를 줄인 것이라는 것을 이해하는 것이 중요합니다. "-다고 해"와 "-대"는 어떤 것을 표현할 때 사용하는 것이기 때문에, "-대"는 자신이 본 것을 말할 때, "-다"는 타인의 언급을 전달할 때 주로 사용됩니다. 따라서 "어째 이상하데?"와 같이 직접적인 언급을 할 때